In [11]:
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install tqdm ipywidgets
#%pip install gymnasium
#%pip install torch

In [12]:
import logging as _logging_
_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

import energyplus.ooep as _ooep_

from energyplus.ooep.addons.display import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

await simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        environment=Model().open(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=Weather().open(_epds_.weathers / 'USA_FL_Tampa.Intl.AP.722110_TMY3.epw'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-8' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:22>>

In [13]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl.gymnasium.spaces import VariableBox
from energyplus.ooep.addons.rl.ray import SimulatorEnv

from energyplus.ooep.components.variables import (
    Actuator,
    OutputVariable,
)
from energyplus.ooep.addons.rl.gymnasium.core import BaseThinEnv

class TestThinEnv(BaseThinEnv):
    @property
    def action_space(self):
        return _gymnasium_.spaces.Dict({
            'thermostat': VariableBox(
                low=20., high=30.,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(Actuator.Specs(
                type='Zone Temperature Control',
                control_type='Cooling Setpoint',
                key='CORE_MID',
            ))
        })
    
    @property
    def observation_space(self):
        return _gymnasium_.spaces.Dict({
            'temperature': VariableBox(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Specs(
                type='People Air Temperature',
                key='CORE_MID',
            )),
        })
    
simulator.add(
    env := TestThinEnv()
)

# TODO mv testing
_ = '''
env.observe()
env.action_space.sample()['thermostat']
env.act(env.action_space.sample())
'''

In [ ]:
simulator
simulator._workflows._handlers
simulator._variables._variable_data

In [ ]:
def _hotfix10447_monkey():
    import energyplus.core
    DataExchange = energyplus.core.pyenergyplus.datatransfer.DataExchange

    if getattr(DataExchange, '_hotfix10447', False):
        return
    
    _get_api_data = DataExchange.get_api_data
    def _hotfix10447_get_api_data(self, state):
        return [
            DataExchange.APIDataExchangePoint(
                _what=datapoint.what,
                _name=datapoint.name, 
                _key=datapoint.type, 
                _type=datapoint.key,
            )
            for datapoint in (
                _get_api_data(self, state)
            )
        ]
    DataExchange.get_api_data = _hotfix10447_get_api_data
    setattr(DataExchange, '_hotfix10447', True)

_hotfix10447_monkey()

In [ ]:
TODO = list(filter(lambda x: x.what == 'OutputVariable', simulator._core.api.exchange.get_api_data(simulator._core.state)))[-1]
TODO.name, TODO.key, TODO.type

('Zone Mechanical Ventilation Air Changes per Hour',
 '',
 'DATACENTER_BASEMENT_ZN_6')

In [ ]:
((simulator.variables.keys()).group(type))

{<class 'energyplus.ooep.components.variables.Clock.Specs'>: <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f68077d5490>, <class 'energyplus.ooep.components.variables.Actuator.Specs'>: <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f67ca8a0a10>, <class 'energyplus.ooep.components.variables.InternalVariable.Specs'>: <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f68003aa8d0>, <class 'energyplus.ooep.components.variables.OutputMeter.Specs'>: <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f67ca8dead0>, <class 'energyplus.ooep.components.variables.OutputVariable.Specs'>: <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f67ca8ddd10>}

In [ ]:
(simulator.variables.keys()).group(type)[_ooep_.components.variables.Actuator.Specs].group(lambda x: x.control_type)

{'Outdoor Dry Bulb': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4130f50>, 'Outdoor Dew Point': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4130fd0>, 'Outdoor Relative Humidity': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4130d90>, 'Diffuse Solar': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4130dd0>, 'Direct Solar': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4130cd0>, 'Wind Speed': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131190>, 'Wind Direction': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131250>, 'Schedule Value': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c41312d0>, 'Surface Property Solar Absorptance': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131350>, 'Surface Property Thermal Absorptance': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131410>, 'Surface Property Visible Absorptance': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131490>, 'Number of People': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131510>, 'Electricity Rate': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131650>, 'Interior Surface Convection Heat Transfer Coefficient': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c41316d0>, 'Exterior Surface Convection Heat Transfer Coefficient': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131790>, 'Construction State': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131810>, 'View Factor To Ground': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131890>, 'Outdoor Air Drybulb Temperature': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131950>, 'Outdoor Air Wetbulb Temperature': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c41319d0>, 'Outdoor Air Wind Speed': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131a50>, 'Outdoor Air Wind Direction': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131b10>, 'Air Exchange Flow Rate': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131b90>, 'On/Off Supervisory': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131c10>, 'Zone Design Heating Air Mass Flow Rate': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131cd0>, 'Zone Design Cooling Air Mass Flow Rate': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131d50>, 'Zone Design Heating Load': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131dd0>, 'Zone Design Cooling Load': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131e90>, 'Zone Design Heating Vol Flow': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131f10>, 'Zone Design Cooling Vol Flow': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4131f90>, 'Heating Setpoint': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4132050>, 'Cooling Setpoint': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c41320d0>, 'Relative Humidity Humidifying Setpoint': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4132150>, 'Relative Humidity Dehumidifying Setpoint': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7fd5c4132210>, 'Mass Flow Rate': <energyplus.ooep.components.variabl